### Functions
API documentation is found here: https://naturstyrelsen.dk/udinaturen/om-udinaturen/

#### getTrips returns dictionary with the next URL and a pandas dataframe the trip/point-data from this offset+limit

In [1]:
import json,urllib.request

def getTrips(url):
    baseUrl = "https://admin.udinaturen.dk"
    dataRaw = urllib.request.urlopen(baseUrl + url).read()
    dataJson = json.loads(dataRaw)

    trips = pd.DataFrame(dataJson["objects"])
    trips.set_index("facilityid", inplace=True)
    
    nexturl = dataJson["meta"]["next"]
    
    return {"nextUrl": nexturl, "trips": trips}

### Fetch online data fra udinaturen... Will take some time
Its only allowed to get 1000 entries at a time. I do 500 instead. Therefore we have to traverse all offset+limits

In [2]:
import pandas as pd

url = ("/api/v1/facilityread?limit=500&offset=0&format=json") # All objects

tripData = getTrips(url) # Get the first 500 trips
allTrips = pd.DataFrame()
while url:
    allTrips = allTrips.append(tripData["trips"])
    url = tripData["nextUrl"]
    if url:
        tripData = getTrips(url) # Get the next 500 trips, if there are more

### Save the data as json for further use

In [3]:
with open('data/rawdata.json', 'w', encoding='utf-8') as file:
    allTrips.to_json(file, force_ascii=False)